# Mock Data Pipeline

Alex Malz

[insert abstract here]

In [ ]:
import numpy as np
import bisect
import astropy.cosmology as cosmology
import scipy.stats as sps
# from scipy.stats import norm
# import emcee
# from datetime import datetime

## Choosing true parameters

[the true redshift-dependent type rate distribution, with plot of three functions]

In [ ]:
types = ['Ia', 'Ibc', 'II']#{'a': 0, 'b': 1, 'c': 2}
n_types = len(types)
frac_types = np.array([1. / n_types] * n_types)
assert np.isclose(np.sum(frac_types), 1.)

n_zs = 100
z_bins = np.linspace(0.01, 1., num=n_zs + 1, endpoint=True)
z_difs = z_bins[1:] - z_bins[:-1]
z_range = np.max(z_bins) - np.min(z_bins)

ns_of_z = []
for t in range(n_types):
    ns_of_z.append(np.ones(n_zs))

true_n_of_z = frac_types[:, np.newaxis] * np.array(ns_of_z) / z_range
assert np.isclose(np.sum(true_n_of_z * z_difs[np.newaxis, :]), 1.)

[samples of t, z from the true redshift-dependent type rate distribution, with histograms]

In [ ]:
def sample_discrete(norm_dist, N):
    out_info = np.empty(shape = (N, 2)).tolist()
    assert np.isclose(np.sum(norm_dist), 1.)
    dist_shape = np.shape(norm_dist)
    flat_dist = norm_dist.flatten()
    cdf = np.cumsum(flat_dist)
    for n in range(N):
        r = np.random.random()
        k = bisect.bisect(cdf, r)
        (t_ind, z_ind) = np.unravel_index(k, shape)
        t = types[t_ind]
        z = np.random.uniform(low=z_bins[z_ind], high=z_bins[z_ind + 1])
        out_info[n][0] = t
        out_info[n][1] = z
    return out_info

n_sne = 10

discrete_n_of_z = true_n_of_z * z_difs[np.newaxis, :]
true_info = sample_discrete(discrete_n_of_z, n_sne)

[the true cosmology]

In [ ]:
H0 = 70
Om0 = 0.3

true_cosmo = cosmology.FlatLambdaCDM(H0=H0, Om0=Om0)

[true mu for each t, z]

In [ ]:
for n in range(n_sne):
    true_info[n].append(true_cosmo.distmod(true_info[n][1]).value)

## Making likelihoods

[the confusion matrix]

In [ ]:
conf_matrix = np.ones((n_types, n_types)) / n_types **2
assert np.isclose(np.sum(conf_matrix, axis=1).all(), frac_types.all())

[set up mu parametrization]

In [ ]:
n_mus = n_zs
mu_bins = np.zeros(n_mus)
for k in range(n_zs):
    mu_bins[k] = true_cosmo.distmod(z_bins[k]).value
mu_difs = mu_bins[1:] - mu_bins[:-1]
mu_range = np.max(mu_bins) - np.min(mu_bins)

z_mids = (z_bins[1:] + z_bins[:-1]) / 2.
mu_mids = (mu_bins[1:] + mu_bins[:-1]) / 2.

z_mu_grid = np.array([[(z, mu) for mu in mu_mids] for z in z_mids])

[the functions taking true type, true z, and true mu returning p(type, z, mu | hat(t))]

In [ ]:
Ia_Ia_var = 1.

In [ ]:
def fit_Ia(z, mu):
    cake = np.zeros((n_types, n_zs, n_mus))
    cake_Ia = sps.multivariate_normal(mean = np.array([z, mu]), cov = Ia_Ia_var * np.eye(2))
    cake[0] = cake_Ia.logpdf(z_mu_grid)
    cake[1] = 
    cake[2] = 
    cake *= conf_matrix[:, 0]
    return cake
    
def fit_Ibc(z, mu):
    
def fit_II(z, mu):
    
def fit_any(true_vals):
    (t, z, mu) = true_vals
    if t == 'Ia':
        cake = fit_Ia(z, mu)
    if t == 'Ibc':
        cake = fit_Ibc(z, mu)
    if t == 'II':
        cake = fit_II(z, mu)
    return cake

def fit_all(catalog):
    dessert = []
    for true_vals in catalog:
        dessert.append(fit_any(true_vals))
    return np.array(dessert)

In [ ]:
def fitter(t_true, t_class, z_true, mu_true):
    if t_true == 'Ia':
        if t_class == 'Ia':
            
        if t_class == 'Ibc':
            
        if t_class == 'II':
        
    if t_true == 'Ibc':
        if t_class == 'Ia':
            
        if t_class == 'Ibc':
            
        if t_class == 'II':
        
    if t_true == 'II':
        if t_class == 'Ia':
            
        if t_class == 'Ibc':
            
        if t_class == 'II':
        

[interim priors to make interim posteriors]

[write data to file]

In [ ]:
def binner(endpoints, items):
    indices = []
    for item in items:
        k = bisect.bisect(endpoints, item)
        indices.append(k)
    return np.array(indices)